<a href="https://colab.research.google.com/github/varekarprajwal/HPCS_LBP/blob/main/HPCS_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title CREATING THE REQUIRED ENVIROMENT
!sudo apt update
!sudo apt install libopencv-dev python3-opencv
!apt-get install -y mpich
!apt-get install -y openmpi-bin openmpi-doc openmpi-dev
!apt-get install -y g++
import numpy as np
import tensorflow as tf

In [ ]:
# @title SELECTING THE PATH
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Project/OPENCV/

Mounted at /content/drive
/content/drive/MyDrive/Project/OPENCV


In [ ]:
# @title SEQUENTIAL CODE
%%writefile Sequential_OPENCV.cpp
#include <iostream>
#include <iomanip>
#include <ctime>
#include <vector>
#include <opencv4/opencv2/opencv.hpp>

using namespace cv;
using namespace std;

// The weight matrix remains a constant global or can be defined locally.
const int wgt[3][3] = {
    {8, 4, 2},
    {16, 0, 1},
    {32, 64, 128}
};

// --- HELPER FUNCTION TO PRINT A CV::MAT ---
void print_matrix(const Mat& mat_to_print, const string& name) {
    cout << "\n--- " << name << " (" << mat_to_print.rows << "x" << mat_to_print.cols << ") ---" << endl;
    for (int i = 0; i < mat_to_print.rows; i++) {
        for (int j = 0; j < mat_to_print.cols; j++) {
            // Use (int) to print the numerical value of the uchar, not the character
            printf("%4d", (int)mat_to_print.at<uchar>(i, j));
        }
        printf("\n");
    }
    cout << "----------------------------------------" << endl;
}


double getCurrentTime() {
    struct timespec currentTime;
    clock_gettime(CLOCK_MONOTONIC, &currentTime);
    return (double)currentTime.tv_sec * 1000.0 + (double)currentTime.tv_nsec / 1000000.0;
}

// Same casecheck function, but now it returns a uchar.
uchar casecheck(uchar a, uchar b, uchar c, uchar d) {
    if (a == b && b == c && c == d) return 7;
    if (a == b) return 1;
    if (b == d) return 2;
    if (c == d) return 3;
    if (a == c) return 4;
    if (a == d) return 5;
    if (c == b) return 6;
    return 0;
}

int main(int argc, char* argv[]) {
    if (argc < 2) {
        cerr << "Usage: " << argv[0] << " <image_path>" << endl;
        return -1;
    }

    Mat image =imread("sample_image/10x10.jpg");
    if (image.empty()) {
        cerr << "Error: Couldn't load input image." << endl;
        return -1;
    }

    Mat hsv_image;
    cvtColor(image, hsv_image, COLOR_BGR2HSV);

    vector<Mat> hsvChannels;
    split(hsv_image, hsvChannels);
    Mat v_channel = hsvChannels[2]; // This is our source image, type CV_8UC1 (uchar)

    const int m_r = v_channel.rows / 2;
    const int m_c = v_channel.cols / 2;

    // Use cv::Mat with the correct uchar type (CV_8UC1) for all images.
    Mat txt_img(m_r, m_c, CV_8UC1);

    // --- Texton Calculation ---
    double startTime1 = getCurrentTime();
    for (int i = 0; i < m_r; ++i) {
        // Get pointers to the two source rows to avoid repeated .at() calls
        const uchar* p_src1 = v_channel.ptr<uchar>(i * 2);
        const uchar* p_src2 = v_channel.ptr<uchar>(i * 2 + 1);
        // Get pointer to the destination row
        uchar* p_dest = txt_img.ptr<uchar>(i);

        for (int j = 0; j < m_c; ++j) {
            uchar a = p_src1[j * 2];
            uchar b = p_src1[j * 2 + 1];
            uchar c = p_src2[j * 2];
            uchar d = p_src2[j * 2 + 1];
            p_dest[j] = casecheck(a, b, c, d);
        }
    }
    double endTime1 = getCurrentTime();

    // --- LTxXORp Calculation ---
    // Initialize the result image by copying the texton image.
    // This efficiently handles the borders, as they remain unchanged.
    Mat main_res = txt_img.clone();

    double startTime2 = getCurrentTime();
    // Loop only over the INTERIOR pixels, from 1 to rows-2 and 1 to cols-2.
    // This avoids the expensive 'if' check for borders inside the loop.
    for (int i = 1; i < m_r - 1; ++i) {
        // Get pointers to the previous, current, and next rows of the texton image
        const uchar* p_prev = txt_img.ptr<uchar>(i - 1);
        const uchar* p_curr = txt_img.ptr<uchar>(i);
        const uchar* p_next = txt_img.ptr<uchar>(i + 1);
        // Get pointer to the destination row in the result matrix
        uchar* p_dest = main_res.ptr<uchar>(i);

        for (int j = 1; j < m_c - 1; ++j) {
            const uchar center_val = p_curr[j];

            // Inlined logic from Texton_weight function, removing function call overhead
            // and the global variable. (condition) evaluates to 0 or 1, creating branchless code.
            int xor_S = 0;
            xor_S += (p_prev[j - 1] != center_val) * wgt[0][0];
            xor_S += (p_prev[j]     != center_val) * wgt[0][1];
            xor_S += (p_prev[j + 1] != center_val) * wgt[0][2];
            xor_S += (p_curr[j - 1] != center_val) * wgt[1][0];
            // xor_S += (p_curr[j]  != center_val) * wgt[1][1]; // Center is always 0
            xor_S += (p_curr[j + 1] != center_val) * wgt[1][2];
            xor_S += (p_next[j - 1] != center_val) * wgt[2][0];
            xor_S += (p_next[j]     != center_val) * wgt[2][1];
            xor_S += (p_next[j + 1] != center_val) * wgt[2][2];

            p_dest[j] = saturate_cast<uchar>(xor_S); // Safely cast to uchar
        }
    }
    double endTime2 = getCurrentTime();

    // --- PRINT THE FINAL RESULT ---
    //print_matrix(txt_img, "Intermediate Texton Image");
    //print_matrix(main_res, "Final LTxXORp Result");


    double totalTime1 = endTime1 - startTime1;
    double totalTime2 = endTime2 - startTime2;

    printf("\n____________________________\n\n");
    printf("Elapsed time for texton : %.4f milliseconds\n", totalTime1);
    printf("Elapsed time for LTxXORp: %.4f milliseconds\n", totalTime2);
    printf("Total elapsed time      : %.4f milliseconds\n", totalTime1 + totalTime2);

    return 0;
}

Overwriting Sequential_OPENCV.cpp


In [ ]:
# @title COMMAND RUN SEQUENTIAL CODE
!g++ Sequential_OPENCV.cpp -o app `pkg-config --cflags --libs opencv4` && ./app

 84  79  54  57  54  60  66  54  80  59 
 74 198 201 211 205 192 197 207 177  78 
 59 200  69  70  72  70  81  55 213  64 
 60 205  72 203 211 203 215  77 200  62 
 57 199  74 210  88  71 203  73 201  68 
 64 190  70 199  67  67 210  75 198  63 
 60 196  82 222 210 212 213  67 204  64 
 56 215  52  76  71  72  74  71 208  60 
 79 176 214 202 203 199 203 205 187  78 
 62  79  63  61  68  62  65  61  80  60 
___________________

Texton image
 
 0   0   0   0   0  
 0   0   0   0   0  
 0   0   3   0   0  
 0   0   0   0   0  
 5   0   0   0   0  
___________________

Texton Weight image
 
  0    0    0    0    0  
  0  128   64   32    0  
  0    1  255   16    0  
  0   34    4    8    0  
  5    0    0    0    0  
Elapsed time for texton : 0.013025 mili seconds
Elapsed time for LTxXORp: 0.031364 mili seconds
Elapsed time: 0.044389 mili seconds


In [ ]:
# @title MPI CODE
%%writefile Parallel_MPI_OPENCV.cpp
#include <iostream>
#include <vector>
#include <numeric>
#include <ctime>
#include <mpi.h>
#include <opencv4/opencv2/opencv.hpp>

using namespace cv;
using namespace std;

// --- HELPER FUNCTION TO PRINT A CV::MAT ---
void print_matrix(const Mat& mat_to_print, const string& name) {
    if (mat_to_print.empty()) {
        cout << "\n--- " << name << " is empty ---" << endl;
        return;
    }
    cout << "\n--- " << name << " (" << mat_to_print.rows << "x" << mat_to_print.cols << ") ---" << endl;
    for (int i = 0; i < mat_to_print.rows; i++) {
        for (int j = 0; j < mat_to_print.cols; j++) {
            // Use (int) to print the numerical value of the uchar, not the character
            printf("%4d", (int)mat_to_print.at<uchar>(i, j));
        }
        printf("\n");
    }
    cout << "----------------------------------------" << endl;
}

// Same casecheck function, but it now returns a uchar (1 byte)
uchar casecheck(uchar a, uchar b, uchar c, uchar d) {
    if (a == b && b == c && c == d) return 7;
    if (a == b) return 1;
    if (b == d) return 2;
    if (c == d) return 3;
    if (a == c) return 4;
    if (a == d) return 5;
    if (c == b) return 6;
    return 0;
}

int main(int argc, char* argv[]) {
    MPI_Init(&argc, &argv);

    int rank, world_size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &world_size);

    if (rank == 0 && argc < 2) {
        cerr << "Usage: mpirun -np <N> " << argv[0] << " <image_path>" << endl;
        MPI_Abort(MPI_COMM_WORLD, 1);
        return -1;
    }

    Mat v_channel_full;
    int full_img_rows = 0, full_img_cols = 0;
    int texton_rows = 0, texton_cols = 0;

    // == 1. Root Process: Load Image and Broadcast Dimensions ==
    if (rank == 0) {
        Mat image =imread("sample_image/10x10.jpg", IMREAD_COLOR);
        if (image.empty()) {
            cerr << "Error: Couldn't load input image." << endl;
            MPI_Abort(MPI_COMM_WORLD, 1);
            return -1;
        }

        Mat hsv_image;
        cvtColor(image, hsv_image, COLOR_BGR2HSV);
        vector<Mat> hsv_planes;
        split(hsv_image, hsv_planes);
        v_channel_full = hsv_planes[2];

        full_img_rows = v_channel_full.rows;
        full_img_cols = v_channel_full.cols;
        texton_rows = full_img_rows / 2;
        texton_cols = full_img_cols / 2;
    }

    // Broadcast essential dimensions to all processes
    MPI_Bcast(&full_img_rows, 1, MPI_INT, 0, MPI_COMM_WORLD);
    MPI_Bcast(&full_img_cols, 1, MPI_INT, 0, MPI_COMM_WORLD);
    MPI_Bcast(&texton_rows, 1, MPI_INT, 0, MPI_COMM_WORLD);
    MPI_Bcast(&texton_cols, 1, MPI_INT, 0, MPI_COMM_WORLD);

    // == 2. Distribute V-Channel Image Data ==
    int base_rows_per_proc = (full_img_rows / 2 / world_size) * 2;

    vector<int> send_counts(world_size);
    vector<int> displacements(world_size);

    if (rank == 0) {
        int current_displacement = 0;
        for (int i = 0; i < world_size; ++i) {
            int rows_to_send = (i < world_size - 1) ? base_rows_per_proc : (full_img_rows - i * base_rows_per_proc);
            send_counts[i] = rows_to_send * full_img_cols;
            displacements[i] = current_displacement;
            current_displacement += send_counts[i];
        }
    }

    MPI_Bcast(send_counts.data(), world_size, MPI_INT, 0, MPI_COMM_WORLD);
    int local_v_rows = send_counts[rank] / full_img_cols;
    Mat local_v_channel(local_v_rows, full_img_cols, CV_8UC1);

    MPI_Scatterv(v_channel_full.data, send_counts.data(), displacements.data(), MPI_UNSIGNED_CHAR,
                 local_v_channel.data, send_counts[rank], MPI_UNSIGNED_CHAR, 0, MPI_COMM_WORLD);

    // == 3. Parallel Texton Calculation ==
    double startTime1 = MPI_Wtime();
    int local_texton_rows = local_v_rows / 2;
    Mat local_texton(local_texton_rows, texton_cols, CV_8UC1);

    for (int i = 0; i < local_texton_rows; ++i) {
        const uchar* p_src1 = local_v_channel.ptr<uchar>(i * 2);
        const uchar* p_src2 = local_v_channel.ptr<uchar>(i * 2 + 1);
        uchar* p_dest = local_texton.ptr<uchar>(i);
        for (int j = 0; j < texton_cols; ++j) {
            p_dest[j] = casecheck(p_src1[j * 2], p_src1[j * 2 + 1], p_src2[j * 2], p_src2[j * 2 + 1]);
        }
    }
    double endTime1 = MPI_Wtime();

    // == 4. Halo Exchange for LTxXORp ==
    Mat local_texton_with_halos(local_texton_rows + 2, texton_cols, CV_8UC1);
    local_texton.copyTo(local_texton_with_halos(Rect(0, 1, texton_cols, local_texton_rows)));

    int prev_rank = (rank == 0) ? MPI_PROC_NULL : rank - 1;
    int next_rank = (rank == world_size - 1) ? MPI_PROC_NULL : rank + 1;

    MPI_Sendrecv(local_texton.ptr<uchar>(0), texton_cols, MPI_UNSIGNED_CHAR, prev_rank, 0,
                 local_texton_with_halos.ptr<uchar>(0), texton_cols, MPI_UNSIGNED_CHAR, prev_rank, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    MPI_Sendrecv(local_texton.ptr<uchar>(local_texton_rows - 1), texton_cols, MPI_UNSIGNED_CHAR, next_rank, 0,
                 local_texton_with_halos.ptr<uchar>(local_texton_rows + 1), texton_cols, MPI_UNSIGNED_CHAR, next_rank, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

    // == 5. Parallel LTxXORp Calculation ==
    double startTime2 = MPI_Wtime();
    Mat local_result = local_texton.clone();
    const int wgt[3][3] = {{8, 4, 2}, {16, 0, 1}, {32, 64, 128}};

    int start_row = (rank == 0) ? 1 : 0;
    int end_row = (rank == world_size - 1) ? local_texton_rows - 1 : local_texton_rows;

    for (int i = start_row; i < end_row; ++i) {
        const uchar* p_prev = local_texton_with_halos.ptr<uchar>(i);
        const uchar* p_curr = local_texton_with_halos.ptr<uchar>(i + 1);
        const uchar* p_next = local_texton_with_halos.ptr<uchar>(i + 2);
        uchar* p_dest = local_result.ptr<uchar>(i);

        for (int j = 1; j < texton_cols - 1; ++j) {
            uchar center_val = p_curr[j];
            int xor_S = 0;
            xor_S += (p_prev[j - 1] != center_val) * wgt[0][0];
            xor_S += (p_prev[j]     != center_val) * wgt[0][1];
            xor_S += (p_prev[j + 1] != center_val) * wgt[0][2];
            xor_S += (p_curr[j - 1] != center_val) * wgt[1][0];
            xor_S += (p_curr[j + 1] != center_val) * wgt[1][2];
            xor_S += (p_next[j - 1] != center_val) * wgt[2][0];
            xor_S += (p_next[j]     != center_val) * wgt[2][1];
            xor_S += (p_next[j + 1] != center_val) * wgt[2][2];
            p_dest[j] = saturate_cast<uchar>(xor_S);
        }
    }
    double endTime2 = MPI_Wtime();

    // == 6. Gather All Results ==
    vector<int> recv_counts(world_size);
    vector<int> texton_displacements(world_size);

    int local_pixel_count = local_texton_rows * texton_cols;
    MPI_Gather(&local_pixel_count, 1, MPI_INT, recv_counts.data(), 1, MPI_INT, 0, MPI_COMM_WORLD);

    Mat texton_full, final_result_full;
    if (rank == 0) {
        texton_displacements[0] = 0;
        for (size_t i = 1; i < recv_counts.size(); ++i) {
            texton_displacements[i] = texton_displacements[i - 1] + recv_counts[i - 1];
        }
        texton_full.create(texton_rows, texton_cols, CV_8UC1);
        final_result_full.create(texton_rows, texton_cols, CV_8UC1);
    }

    MPI_Gatherv(local_texton.data, local_pixel_count, MPI_UNSIGNED_CHAR,
                texton_full.data, recv_counts.data(), texton_displacements.data(),
                MPI_UNSIGNED_CHAR, 0, MPI_COMM_WORLD);

    MPI_Gatherv(local_result.data, local_pixel_count, MPI_UNSIGNED_CHAR,
                final_result_full.data, recv_counts.data(), texton_displacements.data(),
                MPI_UNSIGNED_CHAR, 0, MPI_COMM_WORLD);

    // == 7. Print Timings and Results on Root ==
    double totalTime1 = (endTime1 - startTime1) * 1000.0;
    double totalTime2 = (endTime2 - startTime2) * 1000.0;

    double max_time1, max_time2;
    MPI_Reduce(&totalTime1, &max_time1, 1, MPI_DOUBLE, MPI_MAX, 0, MPI_COMM_WORLD);
    MPI_Reduce(&totalTime2, &max_time2, 1, MPI_DOUBLE, MPI_MAX, 0, MPI_COMM_WORLD);

    if (rank == 0) {
        cout << "\n--- Optimized MPI/OpenCV Execution ---" << endl;
        //print_matrix(texton_full, "Intermediate Texton Image");
        //print_matrix(final_result_full, "Final LTxXORp Result");

        printf("\nMax time for Texton calculation:   %.4f milliseconds\n", max_time1);
        printf("Max time for LTxXORp calculation: %.4f milliseconds\n", max_time2);
        printf("Total max elapsed time:            %.4f milliseconds\n", max_time1 + max_time2);
    }

    MPI_Finalize();
    return 0;
}

Overwriting Parallel_MPI_OPENCV.cpp


In [ ]:
# @title COMMAND RUN PARALLEL MPI CODE
!mpic++ Parallel_MPI_OPENCV.cpp -o app `pkg-config --cflags --libs opencv4` && mpirun --allow-run-as-root -np 1 ./app


GPU HARDWARE ACTIVATED 0 of 1
 84  79  54  57  54  60  66  54  80  59 
 74 198 201 211 205 192 197 207 177  78 
 59 200  69  70  72  70  81  55 213  64 
 60 205  72 203 211 203 215  77 200  62 
 57 199  74 210  88  71 203  73 201  68 
 64 190  70 199  67  67 210  75 198  63 
 60 196  82 222 210 212 213  67 204  64 
 56 215  52  76  71  72  74  71 208  60 
 79 176 214 202 203 199 203 205 187  78 
 62  79  63  61  68  62  65  61  80  60 
___________________

Texton image
 
 0   0   0   0   0  
 0   0   0   0   0  
 0   0   3   0   0  
 0   0   0   0   0  
 5   0   0   0   0  
___________________

Texton Weight image
 
128  64  32  
 1  255  16  
34   4   8  
Time taken by 0 Rank Parallel to calculate Texton code: 0.00101 milliseconds
Time taken by 0 Rank Parallel to calculate LTxXORp code: 0.02734 milliseconds
Elapsed time: 0.028349 mili seconds


In [ ]:
# @title CUDA CODE
%%writefile parallel_CUDA_OPENCV.cu

#include <iostream>
#include <stdio.h>
#include <opencv4/opencv2/core.hpp>
#include <opencv4/opencv2/highgui.hpp>
#include <opencv4/opencv2/opencv.hpp>
#include <cuda_runtime.h>
#include <ctime>

using namespace cv;
using namespace std;

double getCurrentTime()
{
    struct timespec currentTime;
    clock_gettime(CLOCK_MONOTONIC, &currentTime);
    return (double)currentTime.tv_sec * 1000.0 + (double)currentTime.tv_nsec / 1000000.0;
}



__device__ int getGlobalID_3D_3D(){
  int blockId = blockIdx.x + blockIdx.y * gridDim.x+ gridDim.x * gridDim.y * blockIdx.z;
  int threadId = blockId * (blockDim.x * blockDim.y * blockDim.z)+ (threadIdx.z * (blockDim.x * blockDim.y))+ (threadIdx.y * blockDim.x) + threadIdx.x;
  return threadId;
  }

__device__ int casecheck(int a, int b, int c, int d) {
   if (a == b && b == c && c == d && d == a)
        return 7;
    else if (a == b)
        return 1;
    else if (b == d)
        return 2;
    else if (c == d)
        return 3;
    else if (a == c)
        return 4;
    else if (a == d)
        return 5;
    else if (c == b)
        return 6;
    else
        return 0;
}

__global__ void CUDA_START(){
  printf("\nCUDA HARDWARE ACTIVATED\n");
  printf("\n____________________\n");
}
__global__ void calculate_TEXTON_CUDA(int* img_s, int* d_t_img, int rows, int cols)  {
  int x = getGlobalID_3D_3D();
  int a, b, c, d, r, i, q;

  q=x%(cols/2);
  r =x/(cols/2);
  i=((r*2)*cols)+(q*2);
  a = img_s[i], b = img_s[i + 1], c = img_s[cols+i], d = img_s[cols+i+1];

  int rs = casecheck(a, b, c, d);

  d_t_img[x]=rs;
}

__global__ void calculate_LBP_CUDA(int* txt_img, int* Main_res, int rows, const int cols)  {
  int i = getGlobalID_3D_3D();
  //int x =i+(5+1+((i/3)*2));
  int x =i+(cols+1+(i/(cols-2)*2));
  const int a=6,b=5,c=4,d=1,e=1,f=4,g=5,h=6;

   if ( i<(rows*cols)-((rows*2)+(cols*2)-4))
   {
    if (txt_img[x] == txt_img[x-a])//6
      txt_img[x-a] = 0;
    else
      txt_img[x-a] = 1;

    if (txt_img[x] == txt_img[x-b]) //5
      txt_img[x-b] = 0;
    else
      txt_img[x-b] = 1;

    if (txt_img[x] == txt_img[x-c]) //4
      txt_img[x-c] = 0;
    else
      txt_img[x-c] = 1;

    if (txt_img[x] == txt_img[x-d]) //1
      txt_img[x-d] = 0;
    else
      txt_img[x-d] = 1;

    if (txt_img[x] == txt_img[x+e])//1
      txt_img[x+e] = 0;
    else
      txt_img[x+e] = 1;

    if (txt_img[x] == txt_img[x+f]) //4
      txt_img[x+f] = 0;
    else
      txt_img[x+f] = 1;

    if (txt_img[x] == txt_img[x+g]) //5
      txt_img[x+g] = 0;
    else
      txt_img[x+g] = 1;

    if (txt_img[x] == txt_img[x+h]) //6
      txt_img[x+h] = 0;
    else
      txt_img[x+h] = 1;

  int xor_S = 0;
  const int wgt[9] = {8, 4, 2,16, 0, 1,32, 64, 128};
  xor_S =(txt_img[x-a] * wgt[0])+(txt_img[x-b] * wgt[1])+(txt_img[x-c] * wgt[2])+(txt_img[x-d] * wgt[3])+(txt_img[x+e] * wgt[5])+(txt_img[x+f] * wgt[6])+(txt_img[x+g] * wgt[7])+(txt_img[x+h] * wgt[8]);
  Main_res[x]=xor_S;
   }
}

int main() {
  CUDA_START<<<1,1>>>();
  int i, j;
  int t_m_r,t_m_c;
  int i_m_r,i_m_c;
  int *h_img_r = new int[1000*1000];
  Mat image = imread("sample_image/10x10.jpg");

  if (image.empty()) {
    cerr << "Error: Couldn't load input image." << endl;
    return -1;
    }

  Mat hsv_image;
  cvtColor(image, hsv_image, COLOR_BGR2HSV);
  i_m_r = hsv_image.rows ;
  i_m_c = hsv_image.cols ;
  t_m_r = hsv_image.rows / 2;
  t_m_c = hsv_image.cols / 2;


  int k=0;
  for (i = 0; i < hsv_image.rows; i++) {
    for (j = 0; j < hsv_image.cols; j++) {
      Vec3b hsv_pixel = hsv_image.at<Vec3b>(i, j);
      int value = hsv_pixel[2];
      h_img_r[k]=value;
      printf("%3d ", h_img_r[k]);
      k++;
      }
    printf("\n");
  }
  printf("\n____________________\n");



  printf("\nTexton image\n \n");

  int* d_img;
  int* d_t_img;
  int imageSize = i_m_r * i_m_c * sizeof(int);
  int t_imageSize = t_m_r * t_m_c * sizeof(int);

  cudaMalloc((void **)&d_img, imageSize);
  cudaMalloc((void **)&d_t_img, imageSize);


  cudaMemcpy(d_img, h_img_r, imageSize, cudaMemcpyHostToDevice);


  dim3 threadsPerBlock(16, 16);
  dim3 numBlocks1((i_m_c + threadsPerBlock.x - 1) / threadsPerBlock.x, (i_m_r + threadsPerBlock.y - 1) / threadsPerBlock.y);
  dim3 numBlocks2((t_m_c + threadsPerBlock.x - 1) / threadsPerBlock.x, (t_m_r + threadsPerBlock.y - 1) / threadsPerBlock.y);


  double startTime1 = getCurrentTime();

  calculate_TEXTON_CUDA<<<numBlocks1, threadsPerBlock>>>(d_img, d_t_img, i_m_r , i_m_c);

  double endTime1 = getCurrentTime();
  int *h_T_img = new int[t_m_r *t_m_r ];
  cudaMemcpy(h_T_img, d_t_img, t_imageSize, cudaMemcpyDeviceToHost);

  for (i = 0; i < t_m_r; i++) {
    for (j = 0; j < t_m_c; j++) {
      printf("%4d", h_T_img[i*t_m_c+j]);
        }
    printf("\n");
  }

  printf("\n___________________\n");
  printf("\nTexton Weight image\n \n");


  int *h_Txt_img = new int[t_m_r *t_m_r ];
  cudaMalloc((void **)&h_Txt_img, imageSize);

  int *dr_Txt_img = new int[t_m_r *t_m_r ];
  cudaMalloc((void **)&dr_Txt_img, imageSize);

  cudaMemcpy(dr_Txt_img, h_T_img, imageSize, cudaMemcpyHostToDevice);
  cudaMemcpy(h_Txt_img, h_T_img, imageSize, cudaMemcpyHostToDevice);

  double startTime2 = getCurrentTime();

  calculate_LBP_CUDA<<<numBlocks2, threadsPerBlock>>>(dr_Txt_img, h_Txt_img, t_m_r , t_m_c);
  //calculate_LBP_CUDA<<<1,25>>>(dr_Txt_img, h_Txt_img, t_m_r , t_m_c);

  double endTime2 = getCurrentTime();

  int *MM_T_img = new int[t_m_r *t_m_r ];
  cudaMemcpy(MM_T_img, h_Txt_img, t_imageSize, cudaMemcpyDeviceToHost);

  for (i = 0; i < t_m_r; i++) {
    for (j = 0; j < t_m_c; j++) {
      printf("%4d", MM_T_img[i*t_m_c+j]);
        }
    printf("\n");
  }
  double totalTime1 = endTime1 - startTime1;
  printf("Time taken by Parallel to calculate Texton code: %.5f milliseconds\n", totalTime1);
  double totalTime2 = endTime2 - startTime2;
  printf("Time taken by Parallel to calculate LTxXORp code: %.5f milliseconds\n", totalTime2);
  printf("Elapsed time: %f mili seconds\n", totalTime1+totalTime2);

  return 0;
}


Overwriting parallel_CUDA_OPENCV.cu


In [ ]:
# @title COMMAND RUN PARALLEL CUDA CODE
!nvcc -o parallel parallel_CUDA_OPENCV.cu `pkg-config --cflags --libs opencv4` && ./parallel

 84  79  54  57  54  60  66  54  80  59 
 74 198 201 211 205 192 197 207 177  78 
 59 200  69  70  72  70  81  55 213  64 
 60 205  72 203 211 203 215  77 200  62 
 57 199  74 210  88  71 203  73 201  68 
 64 190  70 199  67  67 210  75 198  63 
 60 196  82 222 210 212 213  67 204  64 
 56 215  52  76  71  72  74  71 208  60 
 79 176 214 202 203 199 203 205 187  78 
 62  79  63  61  68  62  65  61  80  60 

____________________

Texton image
 

CUDA HARDWARE ACTIVATED

____________________
   0   0   0   0   0
   0   0   0   0   0
   0   0   3   0   0
   0   0   0   0   0
   5   0   0   0   0

___________________

Texton Weight image
 
   0   0   0   0   0
   0 128  64  32   0
   0   1 255  16   0
   0  34   4   8   0
   5   0   0   0   0
Time taken by Parallel to calculate Texton code: 0.05442 milliseconds
Time taken by Parallel to calculate LTxXORp code: 0.02883 milliseconds
Elapsed time: 0.083247 mili seconds
